Regression

In [13]:
import pandas as pd
import requests
import io

# download boston.csv file
csv_url = "https://ocw.mit.edu/courses/15-071-the-analytics-edge-spring-2017/d4332a3056f44e1a1dec9600a31f21c8_boston.csv"

try:
    response = requests.get(csv_url)
    response.raise_for_status()  # Raise an exception for bad status codes

    df = pd.read_csv(io.StringIO(response.text))
    print(df.head()) # Print first 5 rows of the dataframe

except requests.exceptions.RequestException as e:
    print(f"Error downloading CSV file: {e}")
except pd.errors.ParserError as e:
    print(f"Error parsing CSV data: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


         TOWN  TRACT     LON      LAT  MEDV     CRIM    ZN  INDUS  CHAS  \
0      Nahant   2011 -70.955  42.2550  24.0  0.00632  18.0   2.31     0   
1  Swampscott   2021 -70.950  42.2875  21.6  0.02731   0.0   7.07     0   
2  Swampscott   2022 -70.936  42.2830  34.7  0.02729   0.0   7.07     0   
3  Marblehead   2031 -70.928  42.2930  33.4  0.03237   0.0   2.18     0   
4  Marblehead   2032 -70.922  42.2980  36.2  0.06905   0.0   2.18     0   

     NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  
0  0.538  6.575  65.2  4.0900    1  296     15.3  
1  0.469  6.421  78.9  4.9671    2  242     17.8  
2  0.469  7.185  61.1  4.9671    2  242     17.8  
3  0.458  6.998  45.8  6.0622    3  222     18.7  
4  0.458  7.147  54.2  6.0622    3  222     18.7  


In [14]:
df.dtypes
df_cl = df.drop(columns=['TOWN', 'TRACT'])

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
#import torch.nn.functional as F
#from sklearn.preprocessing import StandardScaler

X = df[['INDUS', 'RM']].to_numpy().astype(np.float32) # Industry level, mean room number
y = df['MEDV'].to_numpy().astype(np.float32).reshape(-1,1) # mean priece

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [19]:
# linear regression
class NN1(nn.Module):
  def __init__(self, input_size) -> None:
      super(NN1, self).__init__()
      self.fc1 = nn.Linear(input_size, 1)

  def forward(self, x):
      x = self.fc1(x)
      return x

# Initioalization
model = NN1(input_size=2)
criterion = nn.MSELoss() # cost function = MSE
optimizer = optim.Adam(model.parameters(), lr=0.01) # Optimizer Adam

# train
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass, jak działa?
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization, update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # co 1000 epok sprawdzam jak działa model:
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [999/10000], Loss: 55.1666
Epoch [1999/10000], Loss: 46.3355
Epoch [2999/10000], Loss: 45.9405
Epoch [3999/10000], Loss: 45.3330
Epoch [4999/10000], Loss: 44.4460
Epoch [5999/10000], Loss: 43.2681
Epoch [6999/10000], Loss: 41.9075
Epoch [7999/10000], Loss: 40.6141
Epoch [8999/10000], Loss: 39.6595
Epoch [9999/10000], Loss: 39.1540
Test Loss: 38.0088


In [20]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.3359,  7.5227]], requires_grad=True)
Parameter containing:
tensor([-20.9828], requires_grad=True)


In [22]:
# Bardziej złożona topologia sieci
class NN2(nn.Module):
  def __init__(self, input_size) -> None:
      super(NN2, self).__init__()
      self.fc1 = nn.Linear(input_size, 10)
      self.fc2 = nn.Linear(10, 8)
      self.fc3 = nn.Linear(8, 1)
      self.relu = nn.ReLU()

  def forward(self, x):
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      x = self.relu(x)
      x = self.fc3(x)
      return x

# Initioalization
model = NN2(input_size=2)
criterion = nn.MSELoss() # cost function = MSE
optimizer = optim.Adam(model.parameters(), lr=0.01) # Optimizer Adam

# train
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass, jak działa?
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization, update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # co 1000 epok sprawdzam jak działa model:
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [1000/10000], Loss: 34.5056
Epoch [2000/10000], Loss: 33.4753
Epoch [3000/10000], Loss: 31.6514
Epoch [4000/10000], Loss: 31.5795
Epoch [5000/10000], Loss: 31.5681
Epoch [6000/10000], Loss: 31.5560
Epoch [7000/10000], Loss: 31.5898
Epoch [8000/10000], Loss: 31.5262
Epoch [9000/10000], Loss: 31.5387
Epoch [10000/10000], Loss: 31.5190
Test Loss: 30.0529


In [23]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.0928, -0.0457],
        [-0.0163,  1.1762],
        [ 0.2786, -0.1949],
        [ 0.0756, -0.7588],
        [ 0.1119,  1.0023],
        [-0.5989,  0.1570],
        [-0.2599,  1.3204],
        [-0.0767, -0.2245],
        [-0.1220,  0.1296],
        [-0.5078, -0.0630]], requires_grad=True)
Parameter containing:
tensor([-0.1037, -0.3828,  1.9589, -0.0682, -3.6832, -0.5911, -3.7640, -0.3635,
         5.8688, -0.2021], requires_grad=True)
Parameter containing:
tensor([[ 0.0924,  0.4783, -0.3999,  0.3061,  2.3366, -0.1057,  1.4505, -0.3127,
         -1.7840,  0.2378],
        [-0.2207,  0.5209,  0.0861, -0.1550,  0.4513, -0.2258,  0.6736, -0.0830,
          0.8876, -0.0097],
        [ 0.0298, -0.3001,  0.0761, -0.1730, -0.1673, -0.2058,  0.1951, -0.0634,
          0.0337, -0.0704],
        [-0.3096, -0.1482, -0.0922, -0.1671, -0.0349, -0.1231,  0.0107,  0.2486,
          0.0936, -0.2584],
        [-0.2228, -0.1031, -0.2242,  0.0750,  0.2439,  0.2831, -0.2559

Classification

In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn

data = load_iris()
X = data['data']
y = data['target']

print(X[:3])
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# konwersja na tensory
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [32]:
class NN3(nn.Module):
  def __init__(self, input_size) -> None:
      super(NN3, self).__init__()
      self.fc1 = nn.Linear(input_size, 3) # bo są 3 klasy

  def forward(self, x):
      x = self.fc1(x)
      return x

# Initioalization
model = NN3(input_size=4)
criterion = nn.CrossEntropyLoss() # Funkcja kosztu CE
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dokładność na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')


# Trenowanie modelu
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass, jak model działa?
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # co 100 epokę spawdzamy jak działą model
    if (epoch+1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss (CE): {test_loss.item():.4f}')

# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.3333
Epoch [100/1000], Loss: 0.8405
Epoch [200/1000], Loss: 0.7695
Epoch [300/1000], Loss: 0.7209
Epoch [400/1000], Loss: 0.6762
Epoch [500/1000], Loss: 0.6361
Epoch [600/1000], Loss: 0.6005
Epoch [700/1000], Loss: 0.5689
Epoch [800/1000], Loss: 0.5407
Epoch [900/1000], Loss: 0.5156
Epoch [1000/1000], Loss: 0.4930
Test Loss (CE): 0.4943
Accuracy: 0.9000


In [33]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.0304,  0.7083, -0.2248, -0.5991],
        [ 0.5677, -0.8218,  0.2380, -0.1011],
        [-0.0595, -0.4706,  0.7621,  0.2719]], requires_grad=True)
Parameter containing:
tensor([ 0.4869,  0.1244, -0.0084], requires_grad=True)


In [34]:
# Bardziej złożony topologicznie model
class NN4(nn.Module):
  def __init__(self, input_size) -> None:
      super(NN4, self).__init__()
      self.fc1 = nn.Linear(input_size, 4)
      self.fc2 = nn.Linear(4, 7)
      self.fc3 = nn.Linear(7, 3)
      self.relu = nn.ReLU()

  def forward(self, x):
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      x = self.relu(x)
      x = self.fc3(x)
      return x

# Initioalization
model = NN4(input_size=4)
criterion = nn.CrossEntropyLoss() # Funkcja kosztu CE
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dokładność na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')


# Trenowanie modelu
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass, jak model działa?
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # co 100 epokę spawdzamy jak działą model
    if (epoch+1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss (CE): {test_loss.item():.4f}')

# Ewaluacja modelu na zbiorze testowym
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.3333
Epoch [100/1000], Loss: 0.8965
Epoch [200/1000], Loss: 0.6583
Epoch [300/1000], Loss: 0.5778
Epoch [400/1000], Loss: 0.4839
Epoch [500/1000], Loss: 0.3716
Epoch [600/1000], Loss: 0.2880
Epoch [700/1000], Loss: 0.2329
Epoch [800/1000], Loss: 0.1956
Epoch [900/1000], Loss: 0.1691
Epoch [1000/1000], Loss: 0.1500
Test Loss (CE): 0.1361
Accuracy: 1.0000


In [35]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[ 0.2814,  0.5352, -0.6371, -0.6470],
        [ 0.2593, -0.1272, -0.5249, -0.0875],
        [-0.2666, -0.1674,  1.0198,  0.8788],
        [ 0.4058,  0.8742, -0.5731, -0.3345]], requires_grad=True)
Parameter containing:
tensor([-0.1492,  0.6666, -1.1619,  0.0907], requires_grad=True)
Parameter containing:
tensor([[ 0.2086, -0.2661, -0.2569, -0.4124],
        [ 0.3683,  0.1621, -0.4618,  0.4662],
        [ 0.6125,  0.5968, -0.5707,  0.1167],
        [ 0.0192,  0.7968, -0.7538,  0.4523],
        [ 0.7229,  0.2021, -1.0105,  0.4756],
        [ 0.6188,  0.6761,  0.5993, -0.5490],
        [ 0.7258,  0.7313,  1.1768, -1.1113]], requires_grad=True)
Parameter containing:
tensor([-0.0252, -0.2665,  0.3552,  0.1286,  0.0190, -0.6014, -0.9205],
       requires_grad=True)
Parameter containing:
tensor([[-0.0256,  0.2686,  0.3612,  0.5092,  0.6708,  0.2234, -0.0930],
        [-0.3236, -0.3062, -0.6965, -0.3459, -0.6656, -0.7922, -0.5356],
        [ 0.1821, -0.4123, -0.40